## LionAGI - 106


Planning & Pydantic Structured Output: Build your own self-optimizing ReAct agent (pt 1)

In [1]:
# %pip install lionagi

In [2]:
from lionagi import __version__

print(__version__)

0.3.4


In [3]:
from IPython.display import display, Markdown

def printmd(string):
    display(Markdown(string))
    
from pathlib import Path
datapath = Path.cwd() / 'data' / "lion-core-source"

### Install LLamaIndex (RAG Tool provider)

In [4]:
# %pip install lionagi llama-index nbconvert

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

docs = SimpleDirectoryReader(
    input_dir=datapath,
    recursive=True,
    required_exts=[".py", ".md", ".ipynb"]
).load_data()

index = VectorStoreIndex.from_documents(documents=docs)
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query(
    "What is the difference between an lion-core Note and a dictionary??",
)

printmd(response.response)

The `Note` class is designed to manage nested dictionary data structures, providing a flexible way to handle complex, hierarchical information. In contrast, a dictionary is a basic data structure that stores key-value pairs without the additional methods and functionalities that the `Note` class offers for manipulating and accessing nested data. Essentially, while a dictionary is a simple container for data, the `Note` class enhances this capability with specialized methods for more sophisticated data management.

In [6]:
from lionfuncs import to_dict

async def query_lion_core(query: str):
    """
    Use this tool to query the lion-core repository for information.
    you should frequently refer to this if you are unsure about anything.
    
    Args:
        query: str: A question, term or query about lion-core.
    """
    response = await query_engine.aquery(query)
    return to_dict(
        response,
        recursive=True,
        recursive_python_only=False,
        max_recursive_depth=4,
    )

---

what is reason action? in terms of AI? And why is it useful?

In tutorial 105, we saw how to use a branch to do function calling. Function Calling is in essence, a type of structured output, and we can do these easier ourselves. 

Let's build our own reason action agent capable of planning using pydantic. A simple illustration of agentic research.

In [7]:
from lionfuncs import to_num
from pydantic import BaseModel, Field


class ReasonModel(BaseModel):
    title: str = Field(
        ...,
        title="Title",
        description="The title of the reason.",
    )
    content: str = Field(
        ...,
        title="Content",
        description="The content that supports or explains the reason.",
    )
    rating: float = Field(
        ...,
        title="Rating",
        description=(
            "The critical objective self rating (1-10) of the reason in terms of correctness."
            " >8 means can continue the path of reasoning, "
            "5-7 you need to recheck your reasoning and consider reversing to previously more confident reason path, "
            "below 5 means you need to stop, must mark more_reason_needed as False, and flag failed as True"
        ),
    )
    more_reason_needed: bool = Field(
        False,
        description=(
            "A flag to indicate whether there are more reasons needed to solve the task."
            "if you are confident that you have enough reasons, set this to False. and stop the reasoning."
            ),)
    
    def is_valid_reason(self):
        try:
            if to_num(self.rating) < 5:
                return False
            if self.content == "":
                return False
            if self.title == "":
                return False
            return True
        except Exception:
            return False

let us try it out

In [8]:
from lionagi import Branch, iModel

system_prompt = """
you are an expert in python and coding, you are asked to provide guidance on how to use the lion-core library.
you have access to the lion-core repository query engine, you can ask questions and get answers.
"""

instruction1 = """
From given context, reason the approach of querying the lion-core repository for information.
"""

In [10]:
branch = Branch(
    imodel=iModel(model="o1-preview", temperature=1),
    tools=query_lion_core,
)

response = await branch.chat(
    instruction=system_prompt+instruction1,
    pydantic_model=ReasonModel,
    return_pydantic_model=True,
)

In [11]:
from lionfuncs import as_readable_json
print(as_readable_json(response.model_dump()))

{
    "title": "Approach to Querying the Lion-Core Repository for Information",
    "content": "To provide guidance on using the lion-core library, I should utilize the lion-core repository query engine efficiently. This involves formulating specific questions about the library's features, usage patterns, and implementation details. By querying for documentation, code examples, and API references within the repository, I can gather accurate and detailed information. This approach ensures that the guidance I provide is based on the most current and relevant data available in the repository.",
    "rating": 9.0,
    "more_reason_needed": false
}


In [12]:
class QueryModel(BaseModel):
    function_name: str = Field(
        ...,
        title="Function Name",
        description="The name of the function to be called. Must pick from given options.",
    )
    query: str = Field(
        ...,
        title="Query",
        description="A detailed precise query to be made to a query engine bot with natural language interface.",
    )


class PlanModel(BaseModel):
    title: str = Field(
        ...,
        title="Title",
        description="The title of the plan.",
    )
    reasons: list[ReasonModel] = Field(
        ...,
        title="Reason",
        description="The reason for the plan.",
    )
    content: str = Field(
        ...,
        title="Content",
        description="The detailed plan content",
    )
    actions: list[QueryModel] = Field(
        default_factory=list,
        title="Actions",
        description="The actions to be taken., You must use action if you don't have information to provide evidence-backed information.",
    )


In [13]:
from lionfuncs import function_to_schema

instruction = """
perform agentic RAG, prepare step by query plan, you should provide 5-10 questions to ask our query engine,
the questions should be logically sequenced and should help in reasoning the approach to build an agent using lion-core.
"""

response = await branch.chat(
    instruction=instruction,
    context={
        "tool_schema": function_to_schema(query_lion_core), 
        "task": "build an agent using lion-core."
    },
    pydantic_model=PlanModel,
    return_pydantic_model=True,
    clear_messages=True,
)

print(as_readable_json(response.model_dump()))

{
    "title": "Plan for Building an Agent Using lion-core",
    "reasons": [
        {
            "title": "Understanding lion-core",
            "content": "To effectively build an agent, it's crucial to understand what lion-core is and its core functionalities.",
            "rating": 5.0,
            "more_reason_needed": false
        },
        {
            "title": "Setting Up the Development Environment",
            "content": "Proper setup ensures all necessary tools and dependencies for lion-core are installed correctly.",
            "rating": 4.8,
            "more_reason_needed": false
        },
        {
            "title": "Learning lion-core's Agent Architecture",
            "content": "Understanding the architecture helps in designing the agent according to best practices.",
            "rating": 5.0,
            "more_reason_needed": false
        },
        {
            "title": "Implementing Custom Agent Behavior",
            "content": "Defining specific be

In [14]:
response.actions

[QueryModel(function_name='query_lion_core', query='What is lion-core and what are its main functionalities?'),
 QueryModel(function_name='query_lion_core', query='How do I install lion-core and what are its prerequisites?'),
 QueryModel(function_name='query_lion_core', query="Where can I find documentation or tutorials on lion-core's agent architecture?"),
 QueryModel(function_name='query_lion_core', query='How do I create a basic agent using lion-core?'),
 QueryModel(function_name='query_lion_core', query='How can I define and implement custom behaviors for an agent in lion-core?'),
 QueryModel(function_name='query_lion_core', query='What are the best practices for testing an agent built with lion-core?'),
 QueryModel(function_name='query_lion_core', query='How do I deploy an agent built with lion-core to a production environment?'),
 QueryModel(function_name='query_lion_core', query='What deployment strategies are recommended for lion-core agents?')]

In [18]:
for idx, item in enumerate(response.actions):
    printmd(f"Query No. {idx}: {item.query} \n --- \n")
    query_response = (await query_lion_core(item.query))["response"]
    printmd(query_response + "\n\n --- \n\n")

Query No. 0: What is lion-core and what are its main functionalities? 
 --- 


Lion-Core is an extraction and refinement of core components from the larger lionagi project, designed to facilitate the development of complex AI and machine learning applications. Its main functionalities are organized into several key components:

1. **Core Components**: Fundamental building blocks for application development, including elements and nodes that can be extended for various purposes.

2. **Data Structures**: Provides essential data management tools such as piles, progressions, and flows to handle data efficiently.

3. **Communication**: Includes systems for messaging and mail management, enabling effective communication within applications.

4. **Utility Systems**: Offers various utility functions, including converters, form validation, and rule systems to support application logic.

5. **AI and ML Tools**: Contains tools for action management and session handling, specifically tailored for AI and machine learning tasks.

These components can be utilized independently or in combination, allowing for modular and scalable application development.

 --- 



Query No. 1: How do I install lion-core and what are its prerequisites? 
 --- 


To install lion-core, you can choose between two options:

1. For the latest stable version, use the following command:
   ```bash
   pip install lion-core
   ```

2. For the latest development version, use:
   ```bash
   pip install git+https://github.com/lion-agi/lion-core.git
   ```

As for prerequisites, it is recommended to set up a virtual environment before installing dependencies. This can be done using the following commands:
```bash
python -m venv venv
source venv/bin/activate  # On Windows use `venv\Scripts\activate`
```

After activating the virtual environment, you can install the necessary dependencies for development.

 --- 



Query No. 2: Where can I find documentation or tutorials on lion-core's agent architecture? 
 --- 


...

The provided information does not include specific best practices for testing an agent built with Lion-Core. It focuses on community engagement, development goals, and the roadmap for enhancing the library. For best practices, it would be advisable to refer to the official documentation or community discussions related to Lion-Core.

 --- 



Query No. 6: How do I deploy an agent built with lion-core to a production environment? 
 --- 


The provided information does not include specific instructions on deploying an agent built with lion-core to a production environment. For deployment guidance, it may be helpful to consult the project's documentation or community resources, such as GitHub Issues or the Discord server, where you can seek support and share experiences with other developers.

 --- 



Query No. 7: What deployment strategies are recommended for lion-core agents? 
 --- 


The provided information does not include any details about deployment strategies for lion-core agents.

 --- 

